# before_remove_TreePosから最終整形を行い，データ拡張実装

In [ ]:
import json

dev = []
with open("data/TreePos_BI/dev.json","r") as f:
    for line in f:
        dev.append(json.loads(line))

for d in dev:
    d["tree_pos"] = [d["tags"][i].split("_")[0] for i in range(len(d["tags"]))]
    d["ner_pos"] = [d["tags"][i].split("_")[1] for i in range(len(d["tags"]))]

In [143]:
import json

train = []
with open("data/TreePos_BI/train.json","r") as f:
    for line in f:
        train.append(json.loads(line))

for d in train:
    d["tree_pos"] = [d["tags"][i].split("_")[0] for i in range(len(d["tags"]))]
    d["ner_pos"] = [d["tags"][i].split("_")[1] for i in range(len(d["tags"]))]

In [37]:
# d= {'tokens': ['The',...,'.'],'tags': ['DT_O',...,'._O'],'tree_pos': ['DT',....'.'], 'ner_pos': ['O',...'O']}
def make_data2list_Simple(data):
    data_list = []
    for d in data:            
        new_sent = []
        before_tag = ""
        new_token = []
        for i in range(len(d["tags"])):

            if d["ner_pos"][i] == "O" or i == len(d["tokens"])-1:#固有表現タグがO　または　最後のトークンの時
                if d["ner_pos"][i] == before_tag:#1
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
                    new_token = []
                else:
                    new_token = []#2
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
                    new_token = []
                    
            else:#固有表現タグがBI#3
                if d["ner_pos"][i] == before_tag:
                    new_token.append(d["tokens"][i])
                    new_token = []
                else:#4
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
            before_tag = d["ner_pos"][i]
        data_list.append(new_sent)
    return data_list
# dev_list= [['Danish', 'rigsdaler'], 'other-currency'],[['='], 'O'],...


def make_data2list_TreePos(data):
    data_list = []
    for d in data:            
        new_sent = []
        before_tag = ""
        new_token = []
        new_tag = []
        for i in range(len(d["tags"])):

            if d["ner_pos"][i] == "O" or i == len(d["tokens"])-1:#固有表現タグがO　または　最後のトークンの時
                if d["ner_pos"][i] == before_tag:#1
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])
                    
                    new_token = []
                    new_tag = []
                else:
                    new_token = []
                    new_tag = []
                    
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])
                    
                    new_token = []
                    new_tag = []
                    
            else:#固有表現タグがBI
                if d["ner_pos"][i] == before_tag:
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_token = []
                    new_tag = []
                else:
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])#new_tagを__で結合
                    
            before_tag = d["ner_pos"][i]
        data_list.append(new_sent)
    return data_list

#交換先辞書の作成
def make_alterna_labels(data_list):
    label_list = set()
    for s in data_list:
        for t in s:
            if t[1] != "O" and "_O" not in t[1][0]:
                label_list.add(t[1])
    
    label_list = list(label_list)
    alterna_labels = dict()
    for label in label_list:
        alterna_labels[label] = set()
    for s in data_list:
        for t in s:
            token = "_".join(t[0])
            tag = t[1]
            if tag in label_list:
                alterna_labels[tag].add(token)
    
    new_alterna_labels = dict()
    for k,v in alterna_labels.items():#候補が空白or1つのtagは削除
        if v==set() or len(v) == 1:
            pass
        else:
            new_alterna_labels[k] = v

    return new_alterna_labels


#Oタグのみデータを排除した増幅候補
def make_data_has_BI(data_list):
    data_has_BI = [] #16342(データ全体は18823)
    for d in data_list:
        tags_list = list(set([i[1] for i in d]))
        if tags_list != ["O"]:
            data_has_BI.append(d)
    return data_has_BI

In [91]:
import random
import json

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

def Simple_TreePos_BI(datatype,data_dict,n):
    #[[token_1,...token_n],tag],...[token_1,...token_n],tag]]状態のデータを作成
    data_simple = make_data2list_Simple(data_dict)#tagが固有表現情報
    data_treepos = make_data2list_TreePos(data_dict)#tagが品詞＋固有表現情報
    
    alterna_labels_simple = make_alterna_labels(data_simple)#固有表現情報のみに着目した交換先辞書
    alterna_labels_treepos = make_alterna_labels(data_treepos)#品詞＋固有表現情報の交換先辞書
    data_has_BI = make_data_has_BI(data_treepos)
    #Oタグのみのデータを排除し，交換前候補として格納（treeposのほうが条件が厳しいのでそれを基準とする）
    
    #加工箇所を同じにするため，SimpleとTreePosを同時に作成する
    data_simple_f_path = "data/Simple_BI_x"+str(n)+"/"+datatype+".json"
    data_treepos_f_path = "data/TreePos_BI_x"+str(n)+"/"+datatype+".json"
    memo_f_path = "log/Simple_and_TreePos_BI_x"+str(n)+"_"+datatype+"_memo.txt"

    with open(data_simple_f_path,"w",encoding="utf-8") as data_simple_f, \
    open(data_treepos_f_path,"w",encoding="utf-8") as data_treepos_f, \
    open(memo_f_path,"w",encoding="utf-8") as memo_f:
        
        for d in data_dict:
            write_d = dict()
            write_d["tokens"] = d["tokens"]
            write_d["tags"] = d["ner_pos"]
            data_simple_f.write(json.dumps(write_d))#Simpleにオリジナルデータを先に書込
            data_simple_f.write("\n")
            data_treepos_f.write(json.dumps(write_d))#TreePosにオリジナルデータを先に書込
            data_treepos_f.write("\n")
        
        add_n = n*len(data_dict)-len(data_dict)#拡張データの数
        add_n
        cnt = 0
        while cnt < add_n:
            if cnt%10 == 0:
                print('\r%d / %d' %(cnt, add_n), end='')

            x = random.randint(0,len(data_has_BI)-1)
            x_s = data_has_BI[x]

            new_d_simple = dict()
            new_d_simple["tokens"] = []
            new_d_simple["tags"] = []
            
            new_d_treepos = dict()
            new_d_treepos["tokens"] = []
            new_d_treepos["tags"] = []
            old_tokens = []
            
            change = False
            print(x_s)
            for token,tag_treepos in x_s:
                tag_simple = tag_treepos.split("_")[-1]
                old_tokens.extend(token)
                #固有表現であり，厳格な候補であるtreeposに交換対象が存在するとき，50%の確率で入れ替えを発生
                if (tag_simple != "O") and (tag_treepos in alterna_labels_treepos) and (random.random() >= 0.5):
                    
                    #固有表現情報に基づく交換候補を生成
                    token_simple = random_choice_token(alterna_labels_simple[tag_simple]-{"_".join(token)})
                    token_simple = token_simple.split("_")
                    #交換したtokenを記録
                    new_d_simple["tokens"].extend(token_simple)
                    #新しいtokenの数だけtagを追加
                    for _ in token_simple:
                        new_d_simple["tags"].append(tag_simple)
                        
                    #品詞＋固有表現情報に基づく交換候補を生成
                    token_treepos = random_choice_token(alterna_labels_treepos[tag_treepos]-{"_".join(token)})
                    token_treepos = token_treepos.split("_")
                    #交換したtokenを記録
                    new_d_treepos["tokens"].extend(token_treepos)
                    #新しいtokenの数だけtagを追加
                    for _ in token_treepos:#
                        new_d_treepos["tags"].append(tag_simple)
                        
                    change = True#書換えフラグ
                    
                else:#alterna_labels_treeposに採用されなかった固有表現タグ付きtoken向けに複数token向けtag追加法で実装
                    new_d_simple["tokens"].extend(token)
                    for _ in token:
                        new_d_simple["tags"].append(tag_simple)

                    new_d_treepos["tokens"].extend(token)
                    for _ in token:
                        new_d_treepos["tags"].append(tag_simple)

            if change == True:#書換えが実行されたなら現在の一文をデータとして書出（改変前と同じ学習データの混入防止）
                if (len(new_d_simple["tokens"]) != len(new_d_simple["tags"])) or (len(new_d_treepos["tokens"]) != len(new_d_treepos["tags"])):
                    print("あかん失敗や")
                data_simple_f.write(json.dumps(new_d_simple))
                data_simple_f.write("\n")
                data_treepos_f.write(json.dumps(new_d_treepos))
                data_treepos_f.write("\n")
                
                memo_f.write(str(x)+"\t"+" ".join(old_tokens)+"\n")#memoに変更前と変更後を記入
                memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d_simple["tokens"])+"\n")
                memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d_treepos["tokens"])+"\n")
                memo_f.write("\n")
                cnt += 1
            else:
                pass
        memo_f.write(str(cnt)+"sentences augmented!!!! \n")

In [92]:
Simple_TreePos_BI("dev",dev,2)

18820 / 18823

In [93]:
Simple_TreePos_BI("train",train,2)

131710 / 131719

# 学習

# DBWS2022ポスター作製

## confusion matrix生成コード

In [ ]:
# !pip3 install gensim
# !pip3 install whatlies

In [26]:
Simple_predictions = []
with open("result/epochs3/Simple_BI_x2/predictions.txt","r") as f:
    for line in f:
        Simple_predictions.extend(line.replace("\n","").split(" "))

In [27]:
TreePos_predictions = []
with open("result/epochs3/TreePos_BI_x2/predictions.txt","r") as f:
    for line in f:
        TreePos_predictions.extend(line.replace("\n","").split(" "))#appendにすると文ごとにリストに格納

In [28]:
import json
true_label = []
with open("data/f/test.json","r") as f:
    for line in f:
        true_label.extend(json.loads(line)["tags"])

Simple_predictions[0:3]

In [34]:
classes = []
with open("data/labels.txt","r") as f:
    for line in f:
        classes.append(line.replace("\n",""))
classes=sorted(set(classes))

In [39]:
type(classes)

list

In [16]:
# !pip install seqeval

In [42]:
from sklearn.metrics import confusion_matrix
val_mat = confusion_matrix(true_label, Simple_predictions, labels=classes)

In [49]:
from sklearn.metrics import confusion_matrix
val_mat2 = confusion_matrix(true_label, TreePos_predictions, labels=classes)

In [43]:
val_mat

array([[712756,     67,     68, ...,    194,    127,    194],
       [    87,   1115,     92, ...,      7,      0,      0],
       [   114,     38,   1669, ...,      0,      0,      0],
       ...,
       [   214,      0,      0, ...,   1072,      0,      1],
       [    78,      0,      0, ...,      1,    595,      0],
       [   238,      0,      6, ...,      0,      7,    911]], dtype=int64)

In [44]:
import numpy
numpy.savetxt("result/epochs3/Simple_BI_x2/confusion_matrix.csv", val_mat, delimiter=",")

In [50]:
numpy.savetxt("result/epochs3/TreePos_BI_x2/confusion_matrix_2.csv", val_mat2, delimiter=",")

## 埋込可視化試み

In [75]:
import json

json_open = open("result/epochs3/Simple_BI_x2/all_results.json","r")
all_result = json.load(json_open)

In [131]:
all_result_d= {}
for k,v in all_result.items():
    k_list = k.split("_")
    tag = k_list[-2]
    parm = k_list[-1]

    if tag not in all_result_d.keys():
        all_result_d[tag] = {}
    if parm in ["f1","number","precision","recall"]:
        all_result_d[tag][parm] = v

In [132]:
label = []
data = []
for k,v in all_result_d.items():
    label.append(k)
    data.append(v)

In [137]:
import csv
with open("result/epochs3/Simple_BI_x2/confusion_matrix_3.csv", 'w') as csvfile:
    writer = csv.DictWriter(csvfile,fieldnames=["f1","number","recall","precision"],lineterminator='\n')
    writer.writeheader()
    for d in data:
        writer.writerow(d)

In [144]:
data_dict = dev+train

In [147]:
data_simple = make_data2list_Simple(data_dict)#tagが固有表現情報
data_treepos = make_data2list_TreePos(data_dict)#tagが品詞＋固有表現情報

alterna_labels_simple = make_alterna_labels(data_simple)#固有表現情報のみに着目した交換先辞書
# alterna_labels_treepos = make_alterna_labels(data_treepos)#品詞＋固有表現情報の交換先辞書
# data_has_BI = make_data_has_BI(data_treepos)

In [155]:
with open("data/alterna_labels_simple.txt","w",encoding="utf-8") as f:
    for k,v in alterna_labels_simple.items():
        f.write("【 "+k+" 】\n")
        f.write("   ".join(list(v))+"\n")#memoに変更前と変更後を記入
        f.write("\n")

In [175]:
new_d = dict()
for k,v in alterna_labels_simple.items():
    words = []
    for token in v:
        words.append(token.replace("_"," "))
    new_d[k] = words

In [178]:
with open("data/alterna_labels_simple_for_dict.txt","w",encoding="utf-8") as f:
    f.write(json.dumps(new_d))

In [14]:
from whatlies import EmbeddingSet
from whatlies.language import HFTransformersLanguage
from whatlies.transformers import Pca

lang = HFTransformersLanguage('bert-base-uncased')

emb = EmbeddingSet(*[lang[w] for w in words])
emb.transform(Pca(2))
emb.plot_interactive()

# [10/8]結果のラベル名を変更（癒着問題解決）

In [194]:
# !pip install evaluate
import json
references = []
with open("data/f/test.json","r",encoding="utf-8") as f:
    for line in f:
        references.append(json.loads(line)["tags"])

predictions = []
with open("result/epochs3/f/predictions.txt","r",encoding="utf-8") as f:
    for line in f:
        predictions.append(line.replace("\n","").split(" "))

label_set = set()
for l in references+predictions:
    for label in list(set(l)):
        label_set.add(label)

label_set.remove("O")
                      
label_d = dict()
for k in label_set:
    label_d[k] = "L"+k.replace("/","_").replace("-","_")
    
for l in references:
    for i in range(len(l)):
        if l[i] in label_d:
            l[i] = label_d[l[i]]

for l in predictions:
    for i in range(len(l)):
        if l[i] in label_d:
            l[i] = label_d[l[i]]

In [195]:
import evaluate
seqeval = evaluate.load('seqeval')
results = seqeval.compute(predictions=predictions, references=references)

In [196]:
results_overall = {}
results_overall['overall_precision']=results['overall_precision']
results_overall['overall_recall']=results['overall_recall']
results_overall['overall_f1']=results['overall_f1']
results_overall['overall_accuracy']=results['overall_accuracy']

del results['overall_precision']
del results['overall_recall']
del results['overall_f1']
del results['overall_accuracy']

result_csv = []
for k in results.keys():
    results[k]["label"] = k
    result_csv.append(results[k])

In [197]:
results_overall

{'overall_precision': 0.6565213121194264,
 'overall_recall': 0.6898412829456564,
 'overall_f1': 0.6727689938708346,
 'overall_accuracy': 0.9277197645306355}

In [22]:
import csv
with open("result/epoch3_f1_results.csv", 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["label","precision","recall","f1","number"],lineterminator='\n')
    writer.writeheader()
    writer.writerows([results[data] for data in results])

## confusion matrix

In [170]:
# !pip install evaluate
import json
references = []
with open("data/f/test.json","r",encoding="utf-8") as f:
    for line in f:
        references.extend(json.loads(line)["tags"])

predictions = []
# with open("result/epochs3/f/predictions.txt","r",encoding="utf-8") as f:
with open("result/epochs3/TreePos_BI_x2/predictions.txt","r",encoding="utf-8") as f:
    for line in f:
        predictions.extend(line.replace("\n","").split(" "))

In [171]:
classes = []
with open("data/labels.txt","r") as f:
    for line in f:
        classes.append(line.replace("\n",""))

In [172]:
from sklearn.metrics import confusion_matrix
val_mat = confusion_matrix(references, predictions, labels=classes)

In [173]:
val_mat

array([[712765,     74,     75, ...,    211,    112,    177],
       [   102,   1109,     94, ...,      6,      0,      0],
       [   101,     61,   1656, ...,      0,      0,      0],
       ...,
       [   218,      0,      0, ...,   1081,      0,      1],
       [    87,      0,      0, ...,      0,    594,      0],
       [   228,      0,      4, ...,      0,      7,    935]], dtype=int64)

In [174]:
import numpy
# numpy.savetxt("result/epochs3_f_confusion_matrix.csv", val_mat, delimiter=",")
numpy.savetxt("result/epochs3_TreePos_BI_x2_confusion_matrix.csv", val_mat, delimiter=",")

## 埋め込みの平均


In [66]:
import json

dev = []
with open("data/f/dev.json","r") as f:
    for line in f:
        dev.append(json.loads(line))

train = []
with open("data/f/train.json","r") as f:
    for line in f:
        train.append(json.loads(line))

In [75]:
# d= {'tokens': ['The',...,'.'],'tags':['O',...'O']}
def make_origin_data2list_Simple(data):
    data_list = []
    for d in data:            
        new_sent = []
        before_tag = ""
        new_token = []
        for i in range(len(d["tags"])):

            if d["tags"][i] == "O" or i == len(d["tokens"])-1:#固有表現タグがO　または　最後のトークンの時
                if d["tags"][i] == before_tag:#1
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["tags"][i]])
                    new_token = []
                else:
                    new_token = []#2
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["tags"][i]])
                    new_token = []
                    
            else:#固有表現タグがBI#3
                if d["tags"][i] == before_tag:
                    new_token.append(d["tokens"][i])
                    new_token = []
                else:#4
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["tags"][i]])
            before_tag = d["tags"][i]
        data_list.append(new_sent)
    return data_list
# dev_list= [['Danish', 'rigsdaler'], 'other-currency'],[['='], 'O'],...


#交換先辞書の作成
def make_alterna_labels(data_list):
    label_list = set()
    for s in data_list:
        for t in s:
            if t[1] != "O" and "_O" not in t[1][0]:
                label_list.add(t[1])
    
    label_list = list(label_list)
    alterna_labels = dict()
    for label in label_list:
        alterna_labels[label] = set()
    for s in data_list:
        for t in s:
            token = " ".join(t[0])
            tag = t[1]
            if tag in label_list:
                alterna_labels[tag].add(token)
    
    new_alterna_labels = dict()
    for k,v in alterna_labels.items():#候補が空白or1つのtagは削除
        if v==set() or len(v) == 1:
            pass
        else:
            new_alterna_labels[k] = v

    return new_alterna_labels

In [76]:
data_simple = make_origin_data2list_Simple(dev+train)#tagが固有表現情報
alterna_labels_simple = make_alterna_labels(data_simple)#固有表現情報のみに着目した交換先辞書
# {'building-theater': {'Hull Truck',
#   'd West',
#   'Théâtre Municipal'...

In [101]:
word = list(alterna_labels_simple['building-theater'])[0]

In [102]:
word

'Hull Truck'

In [120]:
import torch
from transformers import BertModel, BertTokenizer

model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [142]:
alterna_labels_simple.keys()

dict_keys(['building-theater', 'building-sportsfacility', 'art-broadcastprogram', 'product-airplane', 'organization-company', 'person-artist/author', 'organization-showorganization', 'building-hotel', 'event-disaster', 'person-soldier', 'product-other', 'location-other', 'other-currency', 'person-other', 'other-god', 'product-train', 'building-hospital', 'location-mountain', 'organization-other', 'building-restaurant', 'location-park', 'art-music', 'other-award', 'other-chemicalthing', 'person-politician', 'organization-media/newspaper', 'other-astronomything', 'product-food', 'product-weapon', 'organization-education', 'organization-politicalparty', 'person-actor', 'person-athlete', 'product-game', 'location-GPE', 'building-other', 'organization-sportsleague', 'other-biologything', 'location-bodiesofwater', 'other-disease', 'product-ship', 'art-painting', 'event-election', 'organization-government/governmentagency', 'organization-religion', 'other-educationaldegree', 'other-law', 'oth

In [ ]:
new = []
for w in list(alterna_labels_simple["building-theater"]):
    print(w)
    input_ids = torch.tensor(tokenizer.encode(w, add_special_tokens=True)).unsqueeze(0) 
    outputs = model(input_ids)
    new.append(outputs[0].to('cpu').detach().numpy().copy())

In [157]:
import random

x = []
for i in range(10):
    x.append(random.randint(10,99))

In [159]:
for i in range(len(x)):
    if x[i]%3==0:
        print(x[i])

90
84
42


## 埋め込み概念のサイズの話はとん挫

In [164]:
# alterna_labels_simple["location-GPE"]

In [165]:
# s = new[0]
# for w,name in zip(new[1:],list(alterna_labels_simple["building-theater"])[1:]):
#     s = s + w
#     print(name,": ",w.shape)
# s = s / len(new)

In [166]:
# alterna_labels_simple_vec = {}

# for k in alterna_labels_simple.keys():
#     alterna_labels_simple_vec[k] = []
    
#     for w in list(alterna_labels_simple[k]):
#         input_ids = torch.tensor(tokenizer.encode(w, add_special_tokens=True)).unsqueeze(0) 
#         outputs = model(input_ids)
#         alterna_labels_simple_vec[k].append(outputs[0].to('cpu').detach().numpy().copy())

# Oタグのみの判定結果を返した場合の試算

In [185]:
# !pip install evaluate
import json
references = []
with open("data/f/test.json","r",encoding="utf-8") as f:
    for line in f:
        references.append(json.loads(line)["tags"])

predictions_O = []
with open("result/epochs3/f/predictions.txt","r",encoding="utf-8") as f:
    for line in f:
        predictions_O.append(line.replace("\n","").split(" "))

label_set = set()
for l in references+predictions:
    for label in list(set(l)):
        label_set.add(label)

label_set.remove("O")
                      
label_d = dict()
for k in label_set:
    label_d[k] = "L"+k.replace("/","_").replace("-","_")
    
for l in references:
    for i in range(len(l)):
        if l[i] in label_d:
            l[i] = label_d[l[i]]

for l in predictions_O:
    for i in range(len(l)):
        if l[i] in label_d:
            l[i] = ["O" for i in label_d[l[i]]]

In [ ]:
import evaluate
seqeval = evaluate.load('seqeval')
results = seqeval.compute(predictions=predictions_O, references=references)

In [187]:
results_overall = {}
results_overall['overall_precision']=results['overall_precision']
results_overall['overall_recall']=results['overall_recall']
results_overall['overall_f1']=results['overall_f1']
results_overall['overall_accuracy']=results['overall_accuracy']

del results['overall_precision']
del results['overall_recall']
del results['overall_f1']
del results['overall_accuracy']

result_csv = []
for k in results.keys():
    results[k]["label"] = k
    result_csv.append(results[k])

In [188]:
import csv
with open("result/epoch3_f1_results_all_O.csv", 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["label","precision","recall","f1","number"],lineterminator='\n')
    writer.writeheader()
    writer.writerows([results[data] for data in results])

In [192]:
import csv
with open("result/epoch3_f1_results_results_overall_all_O.csv", 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["overall_precision","overall_recall","overall_f1","overall_accuracy"],lineterminator='\n')
    writer.writeheader()
    writer.writerow(results_overall)

In [193]:
results_overall

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.7742944496196749}

# 異なる結果を目視で確認

In [203]:
# !pip install evaluate
import json

references = []
tokens = []
with open("data/f/test.json","r",encoding="utf-8") as f:
    for line in f:
        references.append(json.loads(line)["tags"])
        tokens.append(json.loads(line)["tokens"])
        
predictions = []
with open("result/epochs3/f/predictions.txt","r",encoding="utf-8") as f:
    for line in f:
        predictions.append(line.replace("\n","").split(" "))

label_set = set()
for l in references+predictions:
    for label in list(set(l)):
        label_set.add(label)

label_set.remove("O")
                      
label_d = dict()
for k in label_set:
    label_d[k] = k.replace("/","_").replace("-","_")
    
for l in references:
    for i in range(len(l)):
        if l[i] in label_d:
            l[i] = label_d[l[i]]

for l in predictions:
    for i in range(len(l)):
        if l[i] in label_d:
            l[i] = label_d[l[i]]

In [204]:
miss_result_dict = []
for i in range(len(references)):
    d = {}
    if references[i] != predictions[i]:
        d["tokens"] = tokens[i]
        d["references"] = references[i]
        d["predictions"] = predictions[i]
        miss_result_dict.append(d)

In [205]:
len(miss_result_dict)

20499

In [212]:
with open('result/result_ans_vs_f-model.csv','w',encoding='utf-8') as file:
    for d in miss_result_dict:
        writer = csv.writer(file,delimiter = ",",lineterminator='\n')
        writer.writerows([d["tokens"],d["references"],d["predictions"]])
        file.write("\n")

# Oタグのみでの交換を実施

In [2]:
import json

dev = []
with open("data/TreePos_BI/dev.json","r") as f:
    for line in f:
        dev.append(json.loads(line))

for d in dev:
    d["tree_pos"] = [d["tags"][i].split("_")[0] for i in range(len(d["tags"]))]
    d["ner_pos"] = [d["tags"][i].split("_")[1] for i in range(len(d["tags"]))]

In [3]:
import json

train = []
with open("data/TreePos_BI/train.json","r") as f:
    for line in f:
        train.append(json.loads(line))

for d in train:
    d["tree_pos"] = [d["tags"][i].split("_")[0] for i in range(len(d["tags"]))]
    d["ner_pos"] = [d["tags"][i].split("_")[1] for i in range(len(d["tags"]))]

In [29]:
# d= {'tokens': ['The',...,'.'],'tags': ['DT_O',...,'._O'],'tree_pos': ['DT',....'.'], 'ner_pos': ['O',...'O']}
def make_data2list_Simple(data):
    data_list = []
    for d in data:            
        new_sent = []
        before_tag = ""
        new_token = []
        for i in range(len(d["tags"])):

            if d["ner_pos"][i] == "O" or i == len(d["tokens"])-1:#固有表現タグがO　または　最後のトークンの時
                if d["ner_pos"][i] == before_tag:#1
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
                    new_token = []
                else:
                    new_token = []#2
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
                    new_token = []
                    
            else:#固有表現タグがBI#3
                if d["ner_pos"][i] == before_tag:
                    new_token.append(d["tokens"][i])
                    new_token = []
                else:#4
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
            before_tag = d["ner_pos"][i]
        data_list.append(new_sent)
    return data_list
# dev_list= [['Danish', 'rigsdaler'], 'other-currency'],[['='], 'O'],...


def make_data2list_TreePos(data):
    data_list = []
    for d in data:            
        new_sent = []
        before_tag = ""
        new_token = []
        new_tag = []
        for i in range(len(d["tags"])):

            if d["ner_pos"][i] == "O" or i == len(d["tokens"])-1:#固有表現タグがO　または　最後のトークンの時
                if d["ner_pos"][i] == before_tag:#1
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])
                    
                    new_token = []
                    new_tag = []
                else:
                    new_token = []
                    new_tag = []
                    
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])
                    
                    new_token = []
                    new_tag = []
                    
            else:#固有表現タグがBI
                if d["ner_pos"][i] == before_tag:
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_token = []
                    new_tag = []
                else:
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])#new_tagを__で結合
                    
            before_tag = d["ner_pos"][i]
        data_list.append(new_sent)
    return data_list

#交換先辞書の作成　元の定義名から改変＆名前の変更
def make_alterna_labels_for_O(data_list):
    label_list = set()
    for s in data_list:
        for t in s:
            if t[1] == "O" or "_O" in t[1]:
                label_list.add(t[1])
    
    label_list = list(label_list)
    alterna_labels = dict()
    for label in label_list:
        alterna_labels[label] = set()
    for s in data_list:
        for t in s:
            token = "_".join(t[0])
            tag = t[1]
            if tag in label_list:
                alterna_labels[tag].add(token)
    
    new_alterna_labels = dict()
    for k,v in alterna_labels.items():#候補が空白or1つのtagは削除
        if v==set() or len(v) == 1:
            pass
        else:
            new_alterna_labels[k] = v

    return new_alterna_labels

In [75]:
import random
import json

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

def Simple_TreePos_O(datatype,data_dict,n):#元の定義名から改変＆名前の変更
    
    #[[token_1,...token_n],tag],...[token_1,...token_n],tag]]状態のデータを作成
    data_simple = make_data2list_Simple(data_dict)#tagが固有表現情報
    data_treepos = make_data2list_TreePos(data_dict)#tagが品詞＋固有表現情報
    
    alterna_labels_simple = make_alterna_labels_for_O(data_simple)#固有表現情報のみに着目した交換先辞書
    alterna_labels_treepos = make_alterna_labels_for_O(data_treepos)#品詞＋固有表現情報の交換先辞書
    all_len = len(data_dict)
    #今回は全てOタグを持っている為，make_data_has_BIのOタグ版は使用しない
    
    #加工箇所を同じにするため，SimpleとTreePosを同時に作成する
    data_simple_f_path = "data/Simple_O_x"+str(n)+"/"+datatype+".json"
    data_treepos_f_path = "data/TreePos_O_x"+str(n)+"/"+datatype+".json"
    memo_f_path = "log/Simple_and_TreePos_O_x"+str(n)+"_"+datatype+"_memo.txt"

    with open(data_simple_f_path,"w",encoding="utf-8") as data_simple_f, \
    open(data_treepos_f_path,"w",encoding="utf-8") as data_treepos_f, \
    open(memo_f_path,"w",encoding="utf-8") as memo_f:
        
        for d in data_dict:
            write_d = dict()
            write_d["tokens"] = d["tokens"]
            write_d["tags"] = d["ner_pos"]
            data_simple_f.write(json.dumps(write_d))#Simpleにオリジナルデータを先に書込
            data_simple_f.write("\n")
            data_treepos_f.write(json.dumps(write_d))#TreePosにオリジナルデータを先に書込
            data_treepos_f.write("\n")
        
        add_n = (n-1)*all_len#拡張データの数
        add_n
        cnt = 0
        while cnt < add_n:
            if cnt%10 == 0:
                print('\r%d / %d' %(cnt, add_n), end='')

            x = random.randint(0,all_len-1)
            x_s = data_treepos[x]

            new_d_simple = dict()
            new_d_simple["tokens"] = []
            new_d_simple["tags"] = []
            
            new_d_treepos = dict()
            new_d_treepos["tokens"] = []
            new_d_treepos["tags"] = []
            old_tokens = []
            
            change = False
            for token,tag_treepos in x_s:
                tag_simple = tag_treepos.split("_")[-1]
                old_tokens.extend(token)
                #固有表現であり，厳格な候補であるtreeposに交換対象が存在するとき，50%の確率で入れ替えを発生
                if (tag_simple == "O") and (tag_treepos in alterna_labels_treepos) and (random.random() >= 0.5):
                    
                    #固有表現情報に基づく交換候補を生成
                    token_simple = random_choice_token(alterna_labels_simple[tag_simple]-{"_".join(token)})
                    token_simple = token_simple.split("_")
                    #交換したtokenを記録
                    new_d_simple["tokens"].extend(token_simple)
                    #新しいtokenの数だけtagを追加
                    for _ in token_simple:
                        new_d_simple["tags"].append(tag_simple)
                        
                    #品詞＋固有表現情報に基づく交換候補を生成
                    token_treepos = random_choice_token(alterna_labels_treepos[tag_treepos]-{"_".join(token)})
                    token_treepos = token_treepos.split("_")
                    #交換したtokenを記録
                    new_d_treepos["tokens"].extend(token_treepos)
                    #新しいtokenの数だけtagを追加
                    for _ in token_treepos:#
                        new_d_treepos["tags"].append(tag_simple)
                        
                    change = True#書換えフラグ
                    
                else:#alterna_labels_treeposに採用されなかった固有表現タグ付きtoken向けに複数token向けtag追加法で実装
                    new_d_simple["tokens"].extend(token)
                    for _ in token:
                        new_d_simple["tags"].append(tag_simple)

                    new_d_treepos["tokens"].extend(token)
                    for _ in token:
                        new_d_treepos["tags"].append(tag_simple)

            if change == True:#書換えが実行されたなら現在の一文をデータとして書出（改変前と同じ学習データの混入防止）
                if (len(new_d_simple["tokens"]) != len(new_d_simple["tags"])) or (len(new_d_treepos["tokens"]) != len(new_d_treepos["tags"])):
                    print("あかん失敗や")
                data_simple_f.write(json.dumps(new_d_simple))
                data_simple_f.write("\n")
                data_treepos_f.write(json.dumps(new_d_treepos))
                data_treepos_f.write("\n")
                
                memo_f.write(str(x)+"\t"+" ".join(old_tokens)+"\n")#memoに変更前と変更後を記入
                memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d_simple["tokens"])+"\n")
                memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d_treepos["tokens"])+"\n")
                memo_f.write("\n")
                cnt += 1
            else:
                pass
        memo_f.write(str(cnt)+"sentences augmented!!!! \n")

In [76]:
Simple_TreePos_O("dev",dev,2)

18820 / 18823

In [77]:
Simple_TreePos_O("train",train,2,"str")

131710 / 131719